In [1]:
import pandas as pd
import requests

def get_calories_from_api(ingredient, amount):
    # Replace this URL with the endpoint for your chosen API
    url = "https://api.edamam.com/api/nutrition-data"
    # Add your API key and any other required parameters
    params = {
        'app_id': '716f35ca',
        'app_key': '941ff04905615c00642357ebb6ecb25d	',
        'ingr': f"{amount} {ingredient}"
    }
    
    # Make a request to the API
    response = requests.get(url, params=params)
    data = response.json()
    
    # Check if the API call was successful
    if response.status_code == 200 and 'calories' in data:
        return data['calories']
    else:
        return None

def calculate_calories_for_ingredients(recipe_df):
    # Adding a new column for individual ingredient calories
    recipe_df['Calories'] = recipe_df.apply(
        lambda row: get_calories_from_api(row['Ingredient'], row['Amount']), axis=1
    )
    return recipe_df

# Load your table of recipes
recipe_df = pd.read_csv('recipe.csv')
ingredient, amount = recipe_df["Ingredient"], recipe_df["Amount"]
# Calculate calories for each recipe
recipe_calories_df = calculate_calories_for_ingredients(recipe_df)
print(recipe_calories_df)


/Users/nathanglen/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


                 Ingredient           Amount  Calories
0   White whole wheat flour   2 cups (~240g)       816
1             Baking powder      1 tsp (~5g)         2
2             Ground ginger  1/4 tsp (~0.5g)         1
3                    Nutmeg  1/4 tsp (~0.5g)         2
4                  Cinnamon    1/2 tsp (~1g)         3
5                     Honey  1/3 cup (~113g)       343
6               Coconut oil   1/4 cup (~55g)       486
7                    Butter    5 tbsp (~71g)       509
8                       Egg          1 large        71
9           Vanilla extract      1 tsp (~5g)        12
10           Almond extract      1 tsp (~5g)        12


In [20]:
# # Data for the ingredients and amounts provided by the user
# ingredient_data = {
#     "Ingredient": [
#         "White whole wheat flour", "Baking powder", "Ground ginger", 
#         "Nutmeg", "Cinnamon", "Honey", "Coconut oil", 
#         "Butter", "Egg", "Vanilla extract", "Almond extract"
#     ],
#     "Amount": [
#         "2 cups (~240g)", "1 tsp (~5g)", "1/4 tsp (~0.5g)", 
#         "1/4 tsp (~0.5g)", "1/2 tsp (~1g)", "1/3 cup (~113g)", 
#         "1/4 cup (~55g)", "5 tbsp (~71g)", "1 large", "1 tsp (~5g)", "1 tsp (~5g)"
#     ]
# }
# 
# # Create DataFrame
# ingredient_df = pd.DataFrame(ingredient_data)
# 
# # Save to CSV
# ingredient_csv_path = 'recipe.csv'
# ingredient_df.to_csv(ingredient_csv_path, index=False)
# 
# ingredient_csv_path

In [2]:
import requests

# Set up API credentials and endpoint
API_ID = '716f35ca'
API_KEY = '941ff04905615c00642357ebb6ecb25d'
URL = 'https://api.edamam.com/api/nutrition-details'

def analyze_recipe(ingredients):
    # Prepare payload
    data = {
        "title": "Recipe Analysis",
        "ingr": ingredients  # List of ingredient strings
    }
    
    # Send request to the Edamam API
    response = requests.post(f"{URL}?app_id={API_ID}&app_key={API_KEY}", json=data)
    if response.status_code == 200:
        return response.json()
    else:
        print("Error:", response.status_code)
        return None

# Define compliance check
def check_compliance(nutritional_data):
    total_calories = nutritional_data.get('calories', 0)
    sodium = nutritional_data['totalNutrients'].get('NA', {}).get('quantity', 0)
    total_fat = nutritional_data['totalNutrients'].get('FAT', {}).get('quantity', 0)
    saturated_fat = nutritional_data['totalNutrients'].get('FASAT', {}).get('quantity', 0)
    trans_fat = nutritional_data['totalNutrients'].get('FATRN', {}).get('quantity', 0)
    total_sugars = nutritional_data['totalNutrients'].get('SUGAR', {}).get('quantity', 0)
    recipe_weight = nutritional_data.get('totalWeight', 0)

    compliance = {
        'Calories': total_calories <= 200,
        'Sodium': sodium <= 200,
        'Total Fat': (total_fat * 9) <= 0.35 * total_calories,
        'Saturated Fat': (saturated_fat * 9) < 0.1 * total_calories,
        'Trans Fat': trans_fat == 0,
        'Total Sugars': total_sugars <= 0.35 * recipe_weight
    }
    return compliance

# Example list of ingredients
ingredients_list = [
    "2 cups white whole wheat flour",
    "1 tsp baking powder",
    "1/4 tsp ginger",
    "1/4 tsp nutmeg",
    "1/2 tsp cinnamon",
    "1/3 cup honey",
    "1/4 cup coconut oil, melted",
    "5 Tbsp butter, at room temperature",
    "1 egg",
    "1 tsp vanilla extract",
    "1 tsp almond extract"
]

# Analyze recipe
nutrition_data = analyze_recipe(ingredients_list)

# Check compliance if data is available
if nutrition_data:
    compliance_results = check_compliance(nutrition_data)
    # print("Nutritional Data:", nutrition_data)
    print("Compliance Check:", compliance_results)

Compliance Check: {'Calories': False, 'Sodium': False, 'Total Fat': False, 'Saturated Fat': False, 'Trans Fat': False, 'Total Sugars': True}


In [3]:
import requests

# Set up API credentials and endpoint
API_ID = '716f35ca'
API_KEY = '941ff04905615c00642357ebb6ecb25d'
URL = 'https://api.edamam.com/api/nutrition-details'

def analyze_recipe(ingredients):
    # Prepare payload
    data = {
        "title": "Recipe Analysis",
        "ingr": ingredients  # List of ingredient strings
    }
    
    # Send request to the Edamam API
    response = requests.post(f"{URL}?app_id={API_ID}&app_key={API_KEY}", json=data)
    if response.status_code == 200:
        return response.json()
    else:
        print("Error:", response.status_code)
        return None

# Define compliance check
def check_compliance_per_serving(nutritional_data, servings):
    # Get total values from API response
    total_calories = nutritional_data.get('calories', 0)
    sodium = nutritional_data['totalNutrients'].get('NA', {}).get('quantity', 0)
    total_fat = nutritional_data['totalNutrients'].get('FAT', {}).get('quantity', 0)
    saturated_fat = nutritional_data['totalNutrients'].get('FASAT', {}).get('quantity', 0)
    trans_fat = nutritional_data['totalNutrients'].get('FATRN', {}).get('quantity', 0)
    total_sugars = nutritional_data['totalNutrients'].get('SUGAR', {}).get('quantity', 0)
    recipe_weight = nutritional_data.get('totalWeight', 0)
    
    # Calculate per-serving values
    calories_per_serving = total_calories / servings
    sodium_per_serving = sodium / servings
    total_fat_per_serving = total_fat / servings
    saturated_fat_per_serving = saturated_fat / servings
    trans_fat_per_serving = trans_fat / servings
    total_sugars_per_serving = total_sugars / servings
    weight_per_serving = recipe_weight / servings

    # Check compliance per serving
    compliance = {
        'Calories': calories_per_serving <= 200,
        'Sodium': sodium_per_serving <= 200,
        'Total Fat': (total_fat_per_serving * 9) <= 0.35 * calories_per_serving,
        'Saturated Fat': (saturated_fat_per_serving * 9) < 0.1 * calories_per_serving,
        'Trans Fat': trans_fat_per_serving < 0.5,
        'Total Sugars': total_sugars_per_serving <= 0.35 * weight_per_serving
    }
    
    # Output values for reference
    print(f"Calories per Serving: {calories_per_serving}")
    print(f"Sodium per Serving: {sodium_per_serving}")
    print(f"Total Fat per Serving: {total_fat_per_serving}")
    print(f"Saturated Fat per Serving: {saturated_fat_per_serving}")
    print(f"Trans Fat per Serving: {trans_fat_per_serving}")
    print(f"Total Sugars per Serving: {total_sugars_per_serving}")
    
    return compliance

# Example list of ingredients
ingredients_list = [
    "2 cups white whole wheat flour",
    "1 tsp baking powder",
    "1/4 tsp ginger",
    "1/4 tsp nutmeg",
    "1/2 tsp cinnamon",
    "1/3 cup honey",
    "1/4 cup coconut oil, melted",
    "5 Tbsp butter, at room temperature",
    "1 egg",
    "1 tsp vanilla extract",
    "1 tsp almond extract"
]

# Number of servings
servings = 12

# Analyze recipe
nutrition_data = analyze_recipe(ingredients_list)

# Check compliance if data is available
if nutrition_data:
    compliance_results = check_compliance_per_serving(nutrition_data, servings)
    print("Compliance Check per Serving:", compliance_results)


Calories per Serving: 187.41666666666666
Sodium per Serving: 36.84741666666667
Total Fat per Serving: 10.158696666666668
Saturated Fat per Serving: 6.945349166666667
Trans Fat per Serving: 0.0026333333333333334
Total Sugars per Serving: 7.922521249999999
Compliance Check per Serving: {'Calories': True, 'Sodium': True, 'Total Fat': False, 'Saturated Fat': False, 'Trans Fat': True, 'Total Sugars': True}


In [ ]:
import duckduckgo_search